# Lag Time Analysis 5: Compare xcorr analyses
The values above determine what sort of analysis will be performed.

In [1]:
# Parameters
analysis_tag = "depth-dt-900"
analysis_plotting = False
analysis_data_source = "WBdata-depth.parquet"
analysis_data_source_label = "Depth (m)"
analysis_missing = "full"
analysis_transform = "no_transform"
analysis_filter_type = None
analysis_filter_cutoff = None
analysis_levels = [0, 1, 2, 3, 4, 5, 6, 7]
analysis_search_window = 900
analysis_detrend = True


## Modify four aspects of this procedure to improve performance: 
- STEP 1 - Data source:
    - stream depth (modified from stage)
    - stream discharge
- STEP 2 - Dealing with missing data: 
    - replace missing values with linear interpolation
    - drop missing values
- STEP 3 - Using raw vs. transformed data
    - raw data (no transformation)
    - differenced data (subtract Qi from Qi+1)
    - filtered data (long pass, band pass of various lengths)
- STEP 4 - Split data into shorter segments
    - each 'level' is a factor of two shorter
- STEP 5 - Set Cross-correlation parameters
    - accept all xcorr matches, or only accept within a window


# Preparation

## Imports

In [2]:
%matplotlib inline

import pandas as pd
from pandas.tseries.frequencies import to_offset as pandas_to_offset
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

from scipy.signal import detrend as sig_detrend

import analysis_functions as my

my.print_versions()

Package Versions
----------------
Python: 3.10.16
numpy: 1.26.4
pandas: 2.2.3
scipy: 1.15.1
statsmodels: 0.14.4
pangoin: not imported
matplotlib: 3.10.0
seaborn: 0.13.2
hydrofunctions: 0.2.4
analysis_funtions: 2025.06.27


## Read Data
The data comes from `data_source`

In [3]:
# This dictionary contains all of the parameters for this run
# The values were set in 'LTA_5_XCorr-Execute_Papermill.ipynb', which injected them in the very first cell, above.
# Every time that page is run, the session will be recorded by this page into 'sessions.parquet'.
analysis_start_time = pd.Timestamp.now(tz='utc')
parameters = {
    'analysis_tag': analysis_tag,
    'analysis_start': analysis_start_time.strftime('%Y-%m-%dT%H:%M:%SZ'),
    'analysis_stop': None,
    'analysis_runtime': None,
    'analysis_filename': None,
    'analysis_plotting': analysis_plotting,
    'analysis_data_source': analysis_data_source,
    'analysis_data_source_label': analysis_data_source_label,
    'analysis_missing': analysis_missing,
    'analysis_transform': analysis_transform,
    'analysis_filter_type': analysis_filter_type,
    'analysis_filter_cutoff': analysis_filter_cutoff,
    'analysis_levels': analysis_levels,
    'analysis_search_window': analysis_search_window,
    'analysis_detrend': analysis_detrend,
}

In [4]:
#analysis_data_source = 'WBdata-discharge.parquet'
print(f"Loading data from {analysis_data_source}")
data = pd.read_parquet(analysis_data_source)

# List the first five records. Note that the data are collected every 15 minutes and show discharge in m³/s for each of the four stations.
print(f'First observation: {data.index.min()}')
print(f'Last observation:  {data.index.max()}')
print(f'Length: {data.index.max()-data.index.min()}')
print(f'{data.shape} records')
print(f'Missing: {data.isna().sum()}')
clean_data = data.interpolate()
print(f'Missing after interpolation: {clean_data.isna().sum()}')
clean_data.head()

Loading data from WBdata-depth.parquet


First observation: 2012-10-01 04:00:00+00:00
Last observation:  2016-10-02 03:45:00+00:00
Length: 1461 days 23:45:00
(140352, 5) records
Missing: 01541200    265
01541303     41
01542500      5
01545500     56
01551500     50
dtype: int64
Missing after interpolation: 01541200    0
01541303    0
01542500    0
01545500    0
01551500    0
dtype: int64


01541200  01541303  01542500  01545500  01551500
datetimeUTC                                                                
2012-10-01 04:00:00+00:00  0.514547  0.345172  0.571084  0.730550  0.992743
2012-10-01 04:15:00+00:00  0.517595  0.345172  0.571084  0.730550  0.992743
2012-10-01 04:30:00+00:00  0.514547  0.345172  0.571084  0.724454  0.995791
2012-10-01 04:45:00+00:00  0.514547  0.345172  0.568036  0.724454  0.995791
2012-10-01 05:00:00+00:00  0.514547  0.345172  0.568036  0.721406  0.995791

In [5]:
# Load stations & reach data
stations = pd.read_parquet('stations.parquet') # location of stations on the WB Susquehanna
reaches = pd.read_parquet('reaches.parquet')  # Length of different reaches

#Read validation data
# Validation data. Tables are organized: rows-wave; columns-site or reach.
val_celerity = pd.read_parquet('validation_celerity-table.parquet')
val_time = pd.read_parquet('validation_times-table.parquet')
val_depth = pd.read_parquet('validation_depths-table.parquet')

validation = pd.read_parquet('validation_reaches.parquet') # Organized as records

In [6]:
reaches

us        ds  length       head  gradient  \
Curwensville-Hyde    01541200  01541303   13034   9.309905  0.000714   
Hyde-Karthaus        01541303  01542500   71700  80.918664  0.001129   
Karthaus-Renovo      01542500  01545500   55524  60.320890  0.001086   
Renovo-Williamsport  01545500  01551500   93772  42.537953  0.000454   

                       deltaQ    deltaA symbol    color  
Curwensville-Hyde    1.423295  1.291272      +  #2f9df5  
Hyde-Karthaus        2.834331  3.083876      x  #4df884  
Karthaus-Renovo      2.105634  2.034592      *  #dedd32  
Renovo-Williamsport  1.913043  1.909929      o  #f65f18

## Transform Dataset
If the data will be transformed, then nans will be replaced through linear interpolation first to prevent edge effects.
If the missing data policy is to NOT replace missing data, then the transformed values will be replaced again with Nans.



In [7]:
print(f"Transforming the dataset using the method: {analysis_transform}")

if analysis_transform == 'no_transform':
    print("No transformation will take place.")
    transform_data = clean_data
elif analysis_transform == 'filter':
    # Create a new dataset that has been filtered with a second-order butterworth bandpass filter; run xcorr on this.
    print(f"The filter type is: {analysis_filter_type}")
    print(f"The filter cutoff is: {analysis_filter_cutoff}")
    transform_data = my.butterworth(clean_data, cutoff=analysis_filter_cutoff, sample_period='15 minutes', order=2, filter_type=analysis_filter_type) 
elif analysis_transform == 'difference':
    print("Using a difference filter to transform data.")
    transform_data = clean_data.diff().bfill()
elif analysis_transform == 'smooth-difference':
    smooth_data = my.butterworth(clean_data, cutoff=analysis_filter_cutoff, sample_period='15 minutes', order=2, filter_type='lowpass')
    transform_data = smooth_data.diff().bfill()
else:
    raise ValueError(f"This transform method is not recognized. transform = {analysis_transform}")

missing = transform_data.isna().sum()
print(f'Missing after interpolation: {missing}')
if missing.sum() > 0:
    raise RuntimeError(f"The transformed data contains NaNs. {missing}")

transform_data.head()

Transforming the dataset using the method: no_transform
No transformation will take place.
Missing after interpolation: 01541200    0
01541303    0
01542500    0
01545500    0
01551500    0
dtype: int64


01541200  01541303  01542500  01545500  01551500
datetimeUTC                                                                
2012-10-01 04:00:00+00:00  0.514547  0.345172  0.571084  0.730550  0.992743
2012-10-01 04:15:00+00:00  0.517595  0.345172  0.571084  0.730550  0.992743
2012-10-01 04:30:00+00:00  0.514547  0.345172  0.571084  0.724454  0.995791
2012-10-01 04:45:00+00:00  0.514547  0.345172  0.568036  0.724454  0.995791
2012-10-01 05:00:00+00:00  0.514547  0.345172  0.568036  0.721406  0.995791

In [8]:
%matplotlib inline
# Plot three days of transformed data so we can see what it looks like transformed
fig, (axL, axR) = plt.subplots(ncols=2,figsize=(10, 5), sharex=True)
start = '2015-04-03'
stop = '2015-04-06'
data.loc[start:stop].plot(ax=axL, color=stations['color'])
transform_data.loc[start:stop].plot(ax=axR, color=stations['color'],legend=False)

<Axes: xlabel='datetimeUTC'>

In [9]:
%matplotlib inline
# Plot three days of transformed data so we can see what it looks like transformed
fig, (A,B,C,D,E) = plt.subplots(nrows=5,figsize=(10, 5), sharex=False)

data.plot(ax=A, color=stations['color'])
transform_data.plot(ax=B, color=stations['color'],legend=False)
transform_data.loc['2011-09-30':'2012-11-30'].plot(ax=C, color=stations['color'],legend=False)

%matplotlib inline
start = '2016-08-28'
stop = '2016-09-21'
data.loc[start:stop].plot(ax=D, color=stations['color'],legend=False)
transform_data.loc[start:stop].plot(ax=E, color=stations['color'],legend=False)

<Axes: xlabel='datetimeUTC'>

In [10]:
def length_analysis2(data, upstream, downstream, level, length = None, reach=None, raw_data_label=None, raw_data=None, detrend=False, window=900):
    """Calculate the lag and cross-correlation for a dataset that has been broken into 2^level number of segments.
    
    The purpose of this function is to see how long of a data sequence is needed to produce lags of acceptable quality.
    It will split a dataset into smaller pieces, run xcorr on them, and then record the lag with the best correlation
    into a list, with each element of the list corresponding to one of the pieces of the original dataset.
    
    Inputs:
    =======
    
        data: a dataframe with labelled columns
        upstream: a selection label for a column in data. data[upstream] 
        downstream: the selection label for the downstream station column.
        level: the number of segments that data will be split into, number of segments = 2^level
        length: the distance in meters between the upstream and downstream stations.
        reach: the index value for the reach
        raw_data_label: (str) string to be used as the Y-axis label on plot of the raw data.
        raw_data: If data is transformed for analysis, then this is a dataframe with the untransformed values.
        window: either 'full' for accepting matches of the entire dataset, or a tuple indicating the min and max number of lags in which a search for a match will occur.
        replace_nans: policy for how to replace nans. Uses the options for replace_na()
        plotting (default: True): If true, plot the data in the output.
    
    Returns:
    ========
        Will return 'results', a list of dicts, each dict containing stats about a segment of the data.
        Warning: the US_
    """

    
    if (data.index.freq is None):
        data.index.freq = pd.to_timedelta(pandas_to_offset(pd.infer_freq(data.index)))
        print("new freq: ", data.index.freq)

    num_of_segments = pow(2, level)
    segments = np.array_split(data, num_of_segments)
    seg_length = len(segments[0])
    results = []
    print(f"Analysis from site {upstream} to {downstream} in {num_of_segments} parts; segment length: {seg_length} ({pd.Timedelta('15 minutes') * seg_length})")
    for i, segment_df in enumerate(segments):
        print("\n\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print(f"RUN {i} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        us_df = segment_df[upstream]
        ds_df = segment_df[downstream]

        start = segment_df.index.min() # The start time for this segment.
        stop = segment_df.index.max() # The end time for this segment.
        if raw_data is not None:
            US_nulls = raw_data.loc[start:stop, upstream].isna().sum()
            DS_nulls = raw_data.loc[start:stop, downstream].isna().sum()
        else:
            US_nulls = data.loc[start:stop, upstream].isna().sum()
            DS_nulls = data.loc[start:stop, downstream].isna().sum()
        us_data_max = us_df.max()
        us_data_mean = us_df.mean()
        ds_data_max = ds_df.max()
        ds_data_mean = ds_df.mean()
        old_lag = np.nan
        comments = ""

        # Normalize segment
        us_df = us_df - us_data_mean
        ds_df = ds_df - ds_data_mean

        # Detrend segment
        if detrend == True:
            try:
                #us_df = us_df.interpolate()
                us_df = pd.Series(sig_detrend(us_df), index=segment_df.index) # Keep the same index pd.Series(detrend, 
                us_df.name = f"{upstream}-detrended"
                #ds_df = ds_df.interpolate()
                ds_df = pd.Series(sig_detrend(ds_df), index=segment_df.index)
                ds_df.name = f"{downstream}-detrended"
            except Exception as e:
                print('Error during detrending.')
                print(f"Level: {level}   Segment: {i}   US:{upstream}  DS:{downstream}")
                print(f"missing US values before detrending: {us_df.isna().sum()}")
                print(f"missing DS values before detrending: {ds_df.isna().sum()}")
                print(e)
                raise


        lags, ccor, ccov = my.xcorr_calc(us_df, ds_df)

        # clip the lags & ccor arrays to the window and send to get stats.
        if isinstance(window, int):
            ccor = ccor[lags < window]
            lags = lags[lags < window]
            if len(ccor) < 10 or len(lags) < 10:
                raise RuntimeError(f"After windowing, the length of the lag array and ccov array are too small. len(lags): {len(lags)}  len(ccov): {len(ccov)}")           
            if len(ccor) != len(lags):
                raise RuntimeError(f"After windowing, the length of the lag array and ccov array are different. len(lags): {len(lags)}  len(ccov): {len(ccov)}")
        
        # If the max lag is less than 1, handle it.
        segment_stats = my.xcorr_stats(lags, ccor)
        lag = segment_stats['ccor_max_lag']
        if lag < 0:  # If lag is less than 0, clip the array and try again.
            comments = comments + f"Lag was {lag}. "
            old_lag = lag
            ccor = ccor[lags >= 0]
            lags = lags[lags >= 0]
            segment_stats = my.xcorr_stats(lags, ccor)
            lag = segment_stats['ccor_max_lag']
        
        if lag == 0:
            celerity = np.nan
        else:
            celerity = length / (lag * 15 * 60)

        xc = segment_stats['ccor_max']
        xwidth = segment_stats['1st_0_lag']        
        
        result = {'us': upstream, 'ds': downstream, 'reach': reach, 'level': level, 'segment': i, 
                  'US_nulls': US_nulls, 'DS_nulls': DS_nulls, 'start': start, 'stop': stop, 
                  #'us_data_max': us_data_max, 'us_data_mean': us_data_mean, 
                  #'ds_data_max': ds_data_max, 'ds_data_mean': ds_data_mean,
                  #'us_idxmax': us_idxmax, 'ds_idxmax': ds_idxmax, 'lagmax': lagmax,
                  'lag': lag, 'xcorr': xc, 'xwidth': xwidth, 'celerity': celerity,
                  #'raw_us_data_max': raw_data.loc[start:stop, upstream].max(),
                  #'raw_ds_data_max': raw_data.loc[start:stop, downstream].max(),
                  'raw_us_data_mean': raw_data.loc[start:stop, upstream].mean(),
                  'raw_ds_data_mean': raw_data.loc[start:stop, downstream].mean(),
                  'comments': comments,
                  'old_lag': old_lag,
                 }
        results.append(result)
        print(result)
  
    
    return results

# Cross-Correlation Analysis
Calculate celerity for all four reaches using cross-correlation
run cross-correlation on small chunks of data


In [11]:
#papermill_description=Start_Cross_Correlation
%matplotlib inline

# Set up the analysis by creating a list to hold the results and setting the two stations I'll use for the analysis.
results = []

for level in analysis_levels:
    my.markdown(f"## Level {level}")
    print('*************************************************************')
    print(f'**   Starting Level  {level}                                     **')
    print('*************************************************************\n')
    for reach in reaches.index:
        reach_results = []
        US = reaches.loc[reach, 'us']
        DS = reaches.loc[reach, 'ds']
        length = reaches.loc[reach, 'length']
        print('         ============================================================================')
        print(f'             Starting Level {level}  Reach {reach} => US: {US}   DS: {DS}')
        print('         ============================================================================\n')
        reach_results = length_analysis2(
            transform_data, 
            US, 
            DS, 
            level=level,
            length=length, 
            reach=reach, 
            raw_data_label=analysis_data_source_label,
            raw_data=data, 
            detrend=analysis_detrend,
            window=analysis_search_window
        )
        results = results + reach_results
        print(reach_results)

## Level 0

*************************************************************
**   Starting Level  0                                     **
*************************************************************

             Starting Level 0  Reach Curwensville-Hyde => US: 01541200   DS: 01541303

new freq:  <15 * Minutes>
Analysis from site 01541200 to 01541303 in 1 parts; segment length: 140352 (1462 days 00:00:00)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
RUN 0 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
{'us': '01541200', 'ds': '01541303', 'reach': 'Curwensville-Hyde', 'level': 0, 'segment': 0, 'US_nulls': 265, 'DS_nulls': 41, 'start': Timestamp('2012-10-01 04:00:00+0000', tz='UTC'), 'stop': Timestamp('2016-10-02 03:45:00+0000', tz='UTC'), 'lag': 11, 'xcorr': 0.935591074429819, 'xwidth': nan, 'celerity': 1.3165656565656565, 'raw_us_data_mean': 0.6094345714366066, 'raw_ds_data_mean': 0.6186780128650545, 'comments': '', 'old_lag': nan}
[{'us': '01541200', 'ds': '0154130

C:\Users\marti\anaconda3\envs\py13analysis\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'us': '01545500', 'ds': '01551500', 'reach': 'Renovo-Williamsport', 'level': 0, 'segment': 0, 'US_nulls': 56, 'DS_nulls': 50, 'start': Timestamp('2012-10-01 04:00:00+0000', tz='UTC'), 'stop': Timestamp('2016-10-02 03:45:00+0000', tz='UTC'), 'lag': 43, 'xcorr': 0.9693305627786342, 'xwidth': nan, 'celerity': 2.4230490956072352, 'raw_us_data_mean': 1.1433219435722466, 'raw_ds_data_mean': 1.4927938979306352, 'comments': '', 'old_lag': nan}
[{'us': '01545500', 'ds': '01551500', 'reach': 'Renovo-Williamsport', 'level': 0, 'segment': 0, 'US_nulls': 56, 'DS_nulls': 50, 'start': Timestamp('2012-10-01 04:00:00+0000', tz='UTC'), 'stop': Timestamp('2016-10-02 03:45:00+0000', tz='UTC'), 'lag': 43, 'xcorr': 0.9693305627786342, 'xwidth': nan, 'celerity': 2.4230490956072352, 'raw_us_data_mean': 1.1433219435722466, 'raw_ds_data_mean': 1.4927938979306352, 'comments': '', 'old_lag': nan}]


## Level 1

...

{'us': '01542500', 'ds': '01545500', 'reach': 'Karthaus-Renovo', 'level': 7, 'segment': 116, 'US_nulls': 0, 'DS_nulls': 1, 'start': Timestamp('2016-05-18 04:00:00+0000', tz='UTC'), 'stop': Timestamp('2016-05-29 13:45:00+0000', tz='UTC'), 'lag': 8, 'xcorr': 0.9754173758896165, 'xwidth': 218, 'celerity': 7.711666666666667, 'raw_us_data_mean': 1.1031771405279294, 'raw_ds_data_mean': 1.333260482982855, 'comments': '', 'old_lag': nan}



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
RUN 117 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
{'us': '01542500', 'ds': '01545500', 'reach': 'Karthaus-Renovo', 'level': 7, 'segment': 117, 'US_nulls': 0, 'DS_nulls': 0, 'start': Timestamp('2016-05-29 14:00:00+0000', tz='UTC'), 'stop': Timestamp('2016-06-09 23:45:00+0000', tz='UTC'), 'lag': 38, 'xcorr': 0.9316769640237401, 'xwidth': 114, 'celerity': 1.6235087719298245, 'raw_us_data_mean': 0.9915330164403381, 'raw_ds_data_mean': 1.1322433819929538, 'comments': '', 'old_lag': n

{'us': '01545500', 'ds': '01551500', 'reach': 'Renovo-Williamsport', 'level': 7, 'segment': 16, 'US_nulls': 0, 'DS_nulls': 0, 'start': Timestamp('2013-04-02 00:00:00+0000', tz='UTC'), 'stop': Timestamp('2013-04-13 10:00:00+0000', tz='UTC'), 'lag': 7, 'xcorr': 0.9282550217230443, 'xwidth': 184, 'celerity': 14.884444444444444, 'raw_us_data_mean': 1.6863825929465166, 'raw_ds_data_mean': 2.1046713305204485, 'comments': '', 'old_lag': nan}



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
RUN 17 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
{'us': '01545500', 'ds': '01551500', 'reach': 'Renovo-Williamsport', 'level': 7, 'segment': 17, 'US_nulls': 0, 'DS_nulls': 0, 'start': Timestamp('2013-04-13 10:15:00+0000', tz='UTC'), 'stop': Timestamp('2013-04-24 20:15:00+0000', tz='UTC'), 'lag': 0, 'xcorr': 0.9471982308228022, 'xwidth': 227, 'celerity': nan, 'raw_us_data_mean': 1.765989017741412, 'raw_ds_data_mean': 2.4584477170291086, 'comments': '', 'old_lag': nan}



~~~

{'us': '01545500', 'ds': '01551500', 'reach': 'Renovo-Williamsport', 'level': 7, 'segment': 60, 'US_nulls': 0, 'DS_nulls': 0, 'start': Timestamp('2014-08-17 19:00:00+0000', tz='UTC'), 'stop': Timestamp('2014-08-29 05:00:00+0000', tz='UTC'), 'lag': 42, 'xcorr': 0.9245345002485057, 'xwidth': 226, 'celerity': 2.4807407407407407, 'raw_us_data_mean': 0.9762902830103269, 'raw_ds_data_mean': 1.2764155347137032, 'comments': '', 'old_lag': nan}



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
RUN 61 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
{'us': '01545500', 'ds': '01551500', 'reach': 'Renovo-Williamsport', 'level': 7, 'segment': 61, 'US_nulls': 1, 'DS_nulls': 0, 'start': Timestamp('2014-08-29 05:15:00+0000', tz='UTC'), 'stop': Timestamp('2014-09-09 15:15:00+0000', tz='UTC'), 'lag': 94, 'xcorr': 0.8649609736431851, 'xwidth': 288, 'celerity': 1.1084160756501182, 'raw_us_data_mean': 0.8125454476863846, 'raw_ds_data_mean': 1.0268462949689448, 'comments': '', 'old

{'us': '01545500', 'ds': '01551500', 'reach': 'Renovo-Williamsport', 'level': 7, 'segment': 97, 'US_nulls': 0, 'DS_nulls': 0, 'start': Timestamp('2015-10-14 06:00:00+0000', tz='UTC'), 'stop': Timestamp('2015-10-25 15:45:00+0000', tz='UTC'), 'lag': 123, 'xcorr': 0.8503421889711951, 'xwidth': 268, 'celerity': 0.8470822041553749, 'raw_us_data_mean': 0.7060629002411289, 'raw_ds_data_mean': 0.8244023306867951, 'comments': '', 'old_lag': nan}



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
RUN 98 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
{'us': '01545500', 'ds': '01551500', 'reach': 'Renovo-Williamsport', 'level': 7, 'segment': 98, 'US_nulls': 0, 'DS_nulls': 0, 'start': Timestamp('2015-10-25 16:00:00+0000', tz='UTC'), 'stop': Timestamp('2015-11-06 01:45:00+0000', tz='UTC'), 'lag': 47, 'xcorr': 0.9503951043384616, 'xwidth': 239, 'celerity': 2.2168321513002365, 'raw_us_data_mean': 1.3243758418469687, 'raw_ds_data_mean': 1.5303007759422695, 'comments': '', 'ol

{'us': '01545500', 'ds': '01551500', 'reach': 'Renovo-Williamsport', 'level': 7, 'segment': 126, 'US_nulls': 0, 'DS_nulls': 0, 'start': Timestamp('2016-09-09 08:00:00+0000', tz='UTC'), 'stop': Timestamp('2016-09-20 17:45:00+0000', tz='UTC'), 'lag': 128, 'xcorr': 0.7689622241567327, 'xwidth': 243, 'celerity': 0.8139930555555556, 'raw_us_data_mean': 0.4893100535258006, 'raw_ds_data_mean': 0.5867417613437294, 'comments': '', 'old_lag': nan}



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
RUN 127 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
{'us': '01545500', 'ds': '01551500', 'reach': 'Renovo-Williamsport', 'level': 7, 'segment': 127, 'US_nulls': 0, 'DS_nulls': 0, 'start': Timestamp('2016-09-20 18:00:00+0000', tz='UTC'), 'stop': Timestamp('2016-10-02 03:45:00+0000', tz='UTC'), 'lag': 0, 'xcorr': 0.9423021664286791, 'xwidth': 256, 'celerity': nan, 'raw_us_data_mean': 0.45090414111704147, 'raw_ds_data_mean': 0.5870866080590579, 'comments': '', 'old_lag': nan}

# Results

In [12]:
#papermill_description=Results
# Create a dataframe with the results
resultsDF = pd.DataFrame(results)
# Save the results immediately! But it is temporary-- we don't want to have to re-do the xcorr
resultsDF.to_json(f'temp_save.json', orient='table')

## Calculate residuals (xcorr c - avg validation c)
For each xcorr estimate:
- select all relevant validation observations
- count how many
- calc average celerity
- calc residual (xcorr_c - validation_c)

In [13]:
# Compare each xcorr estimate to the average validation estimate of c and calc a residual
v_time = validation.set_index(['reach', 'DStime']).sort_index()
def mean_count(row):
    try:
        segment = v_time.loc[(row.reach, slice(row.start,row.stop)),:]
        obs_avg = segment['c'].mean()
        lags_avg = segment['dtime'].dt.total_seconds().mean()
        obs_n = segment['c'].count()
    except KeyError as e:
        obs_avg = np.nan
        lags_avg = np.nan
        obs_n = 0
    return pd.Series([obs_avg, lags_avg, obs_n], index=['avg_val_c', 'avg_val_lag', 'val_count'])

new = resultsDF.loc[:,['start', 'stop', 'reach']].apply(mean_count, axis=1)
resultsDF = pd.concat([resultsDF, new], axis=1)
resultsDF['val_count'] = resultsDF['val_count'].astype(int)
resultsDF['resid_c'] = resultsDF['celerity']-resultsDF['avg_val_c']
resultsDF['lag_time'] = resultsDF['lag']*(15 * 60) # 1 lag = 15 min/lag * 60 s/min
resultsDF['resid_lag'] = resultsDF['lag_time']-resultsDF['avg_val_lag']
resultsDF.columns

Index(['us', 'ds', 'reach', 'level', 'segment', 'US_nulls', 'DS_nulls',
       'start', 'stop', 'lag', 'xcorr', 'xwidth', 'celerity',
       'raw_us_data_mean', 'raw_ds_data_mean', 'comments', 'old_lag',
       'avg_val_c', 'avg_val_lag', 'val_count', 'resid_c', 'lag_time',
       'resid_lag'],
      dtype='object')

In [14]:
# Plot of residuals

# Values that origionaly fell outside the window and were re-assigned to the 
# largest peak inside the window are called 're-windowed' and are plotted in red.

re_window = resultsDF.loc[resultsDF['old_lag'].notna()]

fig, ax = plt.subplots()
x1=re_window['celerity']
y1=re_window['avg_val_c']
x=resultsDF['celerity']
y=resultsDF['avg_val_c']
ax.axhline(np.mean(y-x), color='k', linestyle='--')
ax.plot(y,  y-x, 'b.', label="differences")
ax.plot(y1, y1-x1, 'r+', label="re-windowed")
ax.set_yscale('linear')

print(f"Mean difference: {np.mean(y-x)}")
print(f"Mean difference for re-windowed: {np.mean(y1-x1)}")

Mean difference: -2.4453248497251012
Mean difference for re-windowed: 0.36527329635859507


In [15]:
resultsDF[resultsDF['val_count']==0].shape

(326, 23)

## Save Final results to file & Record session

file for the results: "analysis-data-tag-name-.json"

file for parameters of analysis: "analysis-parameters.json"

In [16]:
# Save Session Information
analysis_stop_time = pd.Timestamp.now(tz='UTC')
parameters['analysis_stop'] = analysis_stop_time.strftime('%Y-%m-%dT%H:%M:%SZ')
runtime = (analysis_stop_time - analysis_start_time)
parameters['analysis_runtime'] = str(runtime)
# This name is really long...
parameters['analysis_filename'] = f"xcorr-out-{analysis_tag}-{analysis_stop_time.strftime('%Y-%m-%dT%H')}.json"

# Retrieve old session table and add new line
try:
    old_sessions = pd.read_json('session-parameters.json', orient='table')
except OSError as e:
    old_sessions = pd.DataFrame(columns = parameters.keys())
    old_sessions = old_sessions.astype({'analysis_levels': 'object', 'analysis_search_window': 'object'})

new_sessions = old_sessions._append(parameters, ignore_index=True)    
new_sessions.to_json('session-parameters.json', orient='table')

# Quick read to test
pd.read_json('session-parameters.json', orient='table')

analysis_tag        analysis_start         analysis_stop  \
0         depth  2025-06-28T21:03:33Z  2025-06-28T21:03:52Z   
1     depth-900  2025-06-28T21:03:59Z  2025-06-28T21:04:18Z   
2      depth-dt  2025-06-28T21:04:26Z  2025-06-28T21:04:46Z   
3  depth-dt-900  2025-06-28T21:04:53Z  2025-06-28T21:05:13Z   

         analysis_runtime                          analysis_filename  \
0  0 days 00:00:19.107943         xcorr-out-depth-2025-06-28T21.json   
1  0 days 00:00:18.689082     xcorr-out-depth-900-2025-06-28T21.json   
2  0 days 00:00:19.631976      xcorr-out-depth-dt-2025-06-28T21.json   
3  0 days 00:00:19.932100  xcorr-out-depth-dt-900-2025-06-28T21.json   

  analysis_plotting  analysis_data_source analysis_data_source_label  \
0             False  WBdata-depth.parquet                  Depth (m)   
1             False  WBdata-depth.parquet                  Depth (m)   
2             False  WBdata-depth.parquet                  Depth (m)   
3             False  WBdata-depth.parquet                  Depth (m)   

  analysis_missing analysis_transform  analysis_filter_type  \
0             full       no_transform                   NaN   
1             full       no_transform                   NaN   
2             full       no_transform                   NaN   
3             full       no_transform                   NaN   

   analysis_filter_cutoff           analysis_levels analysis_search_window  \
0                     NaN  [0, 1, 2, 3, 4, 5, 6, 7]                   full   
1                     NaN  [0, 1, 2, 3, 4, 5, 6, 7]                    900   
2                     NaN  [0, 1, 2, 3, 4, 5, 6, 7]                   full   
3                     NaN  [0, 1, 2, 3, 4, 5, 6, 7]                    900   

  analysis_detrend  
0            False  
1            False  
2             True  
3             True

In [17]:
# Save results
resultsDF.to_json(parameters['analysis_filename'], orient='table')
# Read the json file using "orient='table'" to preserve the timestamps.
# The us & ds id's are converted to ints.
# Quick visual check that everything was saved properly
pd.read_json(parameters['analysis_filename'], orient='table')

us        ds                reach  level  segment  US_nulls  \
0     01541200  01541303    Curwensville-Hyde      0        0       265   
1     01541303  01542500        Hyde-Karthaus      0        0        41   
2     01542500  01545500      Karthaus-Renovo      0        0         5   
3     01545500  01551500  Renovo-Williamsport      0        0        56   
4     01541200  01541303    Curwensville-Hyde      1        0       223   
...        ...       ...                  ...    ...      ...       ...   
1015  01545500  01551500  Renovo-Williamsport      7      123         0   
1016  01545500  01551500  Renovo-Williamsport      7      124         0   
1017  01545500  01551500  Renovo-Williamsport      7      125         0   
1018  01545500  01551500  Renovo-Williamsport      7      126         0   
1019  01545500  01551500  Renovo-Williamsport      7      127         0   

      DS_nulls                     start                      stop  lag  ...  \
0           41 2012-10-01 04:00:00+00:00 2016-10-02 03:45:00+00:00   11  ...   
1            5 2012-10-01 04:00:00+00:00 2016-10-02 03:45:00+00:00   41  ...   
2           56 2012-10-01 04:00:00+00:00 2016-10-02 03:45:00+00:00   19  ...   
3           50 2012-10-01 04:00:00+00:00 2016-10-02 03:45:00+00:00   43  ...   
4           22 2012-10-01 04:00:00+00:00 2014-10-02 03:45:00+00:00   11  ...   
...        ...                       ...                       ...  ...  ...   
1015         0 2016-08-06 02:00:00+00:00 2016-08-17 11:45:00+00:00  104  ...   
1016         0 2016-08-17 12:00:00+00:00 2016-08-28 21:45:00+00:00    0  ...   
1017         0 2016-08-28 22:00:00+00:00 2016-09-09 07:45:00+00:00  117  ...   
1018         0 2016-09-09 08:00:00+00:00 2016-09-20 17:45:00+00:00  128  ...   
1019         0 2016-09-20 18:00:00+00:00 2016-10-02 03:45:00+00:00    0  ...   

      raw_us_data_mean  raw_ds_data_mean       comments  old_lag  avg_val_c  \
0             0.609435          0.618678                     NaN   0.896955   
1             0.618678          0.880297                     NaN   1.625949   
2             0.880297          1.143322                     NaN   1.836387   
3             1.143322          1.492794                     NaN   1.587171   
4             0.645802          0.625568                     NaN   0.920016   
...                ...               ...            ...      ...        ...   
1015          0.509027          0.642220                     NaN        NaN   
1016          0.519515          0.711295  Lag was -59.     -59.0        NaN   
1017          0.499797          0.600096                     NaN   0.755008   
1018          0.489310          0.586742                     NaN   0.694607   
1019          0.450904          0.587087                     NaN        NaN   

        avg_val_lag  val_count   resid_c  lag_time     resid_lag  
0      17400.000000         93  0.419610      9900  -7500.000000  
1      49586.746988         83  0.317141     36900 -12686.746988  
2      35032.978723         94  1.410630     17100 -17932.978723  
3      67700.000000         81  0.835878     38700 -29000.000000  
4      17162.068966         29  0.396549      9900  -7262.068966  
...             ...        ...       ...       ...           ...  
1015            NaN          0       NaN     93600           NaN  
1016            NaN          0       NaN         0           NaN  
1017  124200.000000          2  0.135514    105300 -18900.000000  
1018  135000.000000          1  0.119386    115200 -19800.000000  
1019            NaN          0       NaN         0           NaN  

[1020 rows x 23 columns]

## Assert Nulls & Validation Count are correct

Check that the calculations of how many nulls are in the data set are correct, and
check that the correct number of validation observations are in the dataset.

In [18]:
# Check output: amount of missing values (nulls)
# Depth data should show 367 US_nulls and 152 DS_nulls per level and 2936 US_nulls and 1216 DS_nulls for all 8 levels
# Discharge  should show 32062 & 44001 for a single level and 256496 & 352008 for all 8 levels
if analysis_data_source == "WBdata-depth.parquet":
    us = 367
    ds = 152
elif analysis_data_source == "WBdata-discharge.parquet":
    us = 32062
    ds = 44001
elif analysis_data_source == "WBdata-discharge_daily.parquet":
    us = 0
    ds = 0
else:
    print(f"Data source is not recognized. Source: {analysis_data_source}")
    us = 999
    ds = 999

def test_nulls(row, us_nulls, ds_nulls):
    assert row.US_nulls == us_nulls, f"Expected {us_nulls}, but got {row.US_nulls}."
    assert row.DS_nulls == ds_nulls, f"Expected {ds_nulls}, but got {row.DS_nulls}."
    return f"Correct number of nulls. US:{us_nulls}   DS:{ds_nulls}"

print(f"Each level of the analysis should have a total of {us} nulls upstream and {ds} nulls downstream")
resultsDF.groupby(['level'])[['US_nulls', 'DS_nulls']].sum().apply(test_nulls, args=(us,ds), axis=1)

Each level of the analysis should have a total of 367 nulls upstream and 152 nulls downstream


level
0    Correct number of nulls. US:367   DS:152
1    Correct number of nulls. US:367   DS:152
2    Correct number of nulls. US:367   DS:152
3    Correct number of nulls. US:367   DS:152
4    Correct number of nulls. US:367   DS:152
5    Correct number of nulls. US:367   DS:152
6    Correct number of nulls. US:367   DS:152
7    Correct number of nulls. US:367   DS:152
dtype: object

In [19]:
# Check output: validation data count
# There should be 93, 83, 94, 81 = 351 validation observations in each level.
def test_v_obs(row):
     #print(row)
    Level, Reach = row.name
    if Reach == 'Curwensville-Hyde':
        v_count = 93
    elif Reach == 'Hyde-Karthaus':
        v_count = 83
    elif Reach == 'Karthaus-Renovo':
        v_count = 94
    elif Reach == 'Renovo-Williamsport':
        v_count = 81
    #assert row.val_count == v_count, f"Was expecting {Level}-{Reach} val_count to equal {v_count} observations, but got {row.val_count}."
    return f"{row.val_count}"# is good"

print("There should be 93, 83, 94, 81 = 351 validation observations in each level.")
(resultsDF.groupby(['level', 'reach'])[['US_nulls','DS_nulls','val_count']]
     .sum()
     .apply(test_v_obs, axis=1)
     .unstack('reach')
)

There should be 93, 83, 94, 81 = 351 validation observations in each level.


reach Curwensville-Hyde Hyde-Karthaus Karthaus-Renovo Renovo-Williamsport
level                                                                    
0                    93            83              94                  81
1                    93            83              94                  81
2                    93            83              94                  81
3                    93            83              94                  81
4                    93            83              94                  81
5                    93            83              94                  81
6                    93            83              94                  81
7                    93            83              94                  81

# Evaluation of Results

## View the Results

In [20]:
resultsDF.describe()

level      segment     US_nulls     DS_nulls          lag  \
count  1020.000000  1020.000000  1020.000000  1020.000000  1020.000000   
mean      6.031373    42.333333     2.878431     1.192157    42.785294   
std       1.322781    36.048526    17.268387     4.522579    98.616514   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       6.000000    12.000000     0.000000     0.000000     7.000000   
50%       7.000000    32.000000     0.000000     0.000000    19.000000   
75%       7.000000    64.000000     1.000000     0.000000    44.000000   
max       7.000000   127.000000   265.000000    56.000000   899.000000   

             xcorr      xwidth    celerity  raw_us_data_mean  \
count  1020.000000  928.000000  816.000000       1020.000000   
mean      0.863225  269.830819    4.084707          0.812932   
std       0.163242  148.309199   10.553980          0.381564   
min       0.134805    2.000000    0.016109          0.179495   
25%       0.843066  179.000000    1.105278          0.538783   
50%       0.923570  228.000000    1.695035          0.726557   
75%       0.962341  331.250000    2.804242          1.032820   
max       0.994706  876.000000  104.191111          2.797250   

       raw_ds_data_mean      old_lag   avg_val_c    avg_val_lag    val_count  \
count       1020.000000    59.000000  694.000000     694.000000  1020.000000   
mean           1.033779  -256.644068    1.471337   42762.887047     2.752941   
std            0.528215   299.905546    0.643989   24650.295972     7.582438   
min            0.214097 -1195.000000    0.212974    6300.000000     0.000000   
25%            0.645170  -433.500000    0.948413   22286.250000     0.000000   
50%            0.943378  -114.000000    1.450371   39600.000000     1.000000   
75%            1.337995   -26.000000    1.810606   55800.000000     2.000000   
max            3.704354    -1.000000    4.341296  153900.000000    94.000000   

          resid_c       lag_time      resid_lag  
count  607.000000    1020.000000     694.000000  
mean     2.445325   38506.764706   -8217.930275  
std      9.381546   88754.862717   77814.785705  
min     -2.248075       0.000000  -97200.000000  
25%      0.149647    6300.000000  -26100.000000  
50%      0.447391   17100.000000  -12195.378151  
75%      1.188425   39600.000000   -5400.000000  
max    102.560551  809100.000000  791100.000000

In [21]:
print(resultsDF.shape)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(resultsDF)

(1020, 23)


us        ds                reach  level  segment  US_nulls  \
0     01541200  01541303    Curwensville-Hyde      0        0       265   
1     01541303  01542500        Hyde-Karthaus      0        0        41   
2     01542500  01545500      Karthaus-Renovo      0        0         5   
3     01545500  01551500  Renovo-Williamsport      0        0        56   
4     01541200  01541303    Curwensville-Hyde      1        0       223   
5     01541200  01541303    Curwensville-Hyde      1        1        42   
6     01541303  01542500        Hyde-Karthaus      1        0        22   
7     01541303  01542500        Hyde-Karthaus      1        1        19   
8     01542500  01545500      Karthaus-Renovo      1        0         4   
9     01542500  01545500      Karthaus-Renovo      1        1         1   
10    01545500  01551500  Renovo-Williamsport      1        0        31   
11    01545500  01551500  Renovo-Williamsport      1        1        25   
12    01541200  01541303    Curwensville-Hyde      2        0       204   
13    01541200  01541303    Curwensville-Hyde      2        1        19   
14    01541200  01541303    Curwensville-Hyde      2        2        18   
15    01541200  01541303    Curwensville-Hyde      2        3        24   
16    01541303  01542500        Hyde-Karthaus      2        0        22   
17    01541303  01542500        Hyde-Karthaus      2        1         0   
18    01541303  01542500        Hyde-Karthaus      2        2        13   
19    01541303  01542500        Hyde-Karthaus      2        3         6   
20    01542500  01545500      Karthaus-Renovo      2        0         1   
21    01542500  01545500      Karthaus-Renovo      2        1         3   
22    01542500  01545500      Karthaus-Renovo      2        2         1   
23    01542500  01545500      Karthaus-Renovo      2        3         0   
24    01545500  01551500  Renovo-Williamsport      2        0        15   
25    01545500  01551500  Renovo-Williamsport      2        1        16   
26    01545500  01551500  Renovo-Williamsport      2        2        16   
27    01545500  01551500  Renovo-Williamsport      2        3         9   
28    01541200  01541303    Curwensville-Hyde      3        0        12   
29    01541200  01541303    Curwensville-Hyde      3        1       192   
30    01541200  01541303    Curwensville-Hyde      3        2        12   
31    01541200  01541303    Curwensville-Hyde      3        3         7   
32    01541200  01541303    Curwensville-Hyde      3        4        14   
33    01541200  01541303    Curwensville-Hyde      3        5         4   
34    01541200  01541303    Curwensville-Hyde      3        6        10   
35    01541200  01541303    Curwensville-Hyde      3        7        14   
36    01541303  01542500        Hyde-Karthaus      3        0         2   
37    01541303  01542500        Hyde-Karthaus      3        1        20   
38    01541303  01542500        Hyde-Karthaus      3        2         0   
39    01541303  01542500        Hyde-Karthaus      3        3         0   
40    01541303  01542500        Hyde-Karthaus      3        4        12   
41    01541303  01542500        Hyde-Karthaus      3        5         1   
42    01541303  01542500        Hyde-Karthaus      3        6         6   
43    01541303  01542500        Hyde-Karthaus      3        7         0   
44    01542500  01545500      Karthaus-Renovo      3        0         1   
45    01542500  01545500      Karthaus-Renovo      3        1         0   
46    01542500  01545500      Karthaus-Renovo      3        2         1   
47    01542500  01545500      Karthaus-Renovo      3        3         2   
48    01542500  01545500      Karthaus-Renovo      3        4         1   
49    01542500  01545500      Karthaus-Renovo      3        5         0   
50    01542500  01545500      Karthaus-Renovo      3        6         0   
51    01542500  01545500      Karthaus-Renovo      3        7         0   
52    01545500  01551500  Renovo-Will